In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
import optuna
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegressionCV
import joblib
from boruta import BorutaPy
from lightgbm import LGBMClassifier
import lightgbm as lgb

/Users/prathmesh/asep_project/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load both datasets
original = pd.read_csv("PCOS_data.csv")
new = pd.read_csv("pcos_dataset.csv")

# Set max columns to show is unlimited
pd.set_option('display.max_columns', None)

In [3]:
def preprocess(df):
    df = df.copy()  # avoid SettingWithCopyWarning

    # 1. Clean column names
    df.columns = df.columns.str.strip() \
                           .str.replace(' ', '_') \
                           .str.replace('(', '') \
                           .str.replace(')', '') \
                           .str.replace('.', '') \
                           .str.replace('-', '_') \
                           .str.replace('/', '_')
    df.rename(columns={'II____beta_HCGmIU_mL': 'II_beta_HCG'}, inplace=True)

    # 2. Drop irrelevant columns
    df.drop(columns=['Sl_No', 'Patient_File_No'], inplace=True, errors='ignore')
    df = df.loc[:, ~df.columns.str.startswith('Unnamed')]

    # 3. Merge Age columns
    if 'Age' not in df.columns and 'Age_yrs' in df.columns:
        df.rename(columns={'Age_yrs': 'Age'}, inplace=True)
    elif 'Age' in df.columns and 'Age_yrs' in df.columns:
        df['Age'] = df['Age'].fillna(df['Age_yrs'])
        df.drop(columns=['Age_yrs'], inplace=True)

    # 4. Merge PCOS diagnosis columns
    if 'PCOS_Diagnosis' in df.columns:
        df.rename(columns={'PCOS_Diagnosis': 'PCOS_Y_N'}, inplace=True)

    # 5. Handle missing values
    if 'Marraige_Status_Yrs' in df.columns:
        df.loc[:, 'Marraige_Status_Yrs'] = df['Marraige_Status_Yrs'].fillna(df['Marraige_Status_Yrs'].median())

    if 'Fast_food_Y_N' not in df.columns and 'Fast_food_YN' in df.columns:
        df.rename(columns={'Fast_food_YN': 'Fast_food_Y_N'}, inplace=True)
    if 'Fast_food_Y_N' in df.columns:
        df.loc[:, 'Fast_food_Y_N'] = df['Fast_food_Y_N'].fillna(df['Fast_food_Y_N'].mode()[0])

    # 6. Convert to numeric and fill missing values
    if 'II_beta_HCG' not in df.columns and 'II_beta_HCGmIU_mL' in df.columns:
        df.rename(columns={'II_beta_HCGmIU_mL': 'II_beta_HCG'}, inplace=True)
    if 'II_beta_HCG' in df.columns:
        df.loc[:, 'II_beta_HCG'] = pd.to_numeric(df['II_beta_HCG'], errors='coerce')
        df['II_beta_HCG'] = df['II_beta_HCG'].astype(float)
        df.loc[:, 'II_beta_HCG'] = df['II_beta_HCG'].fillna(df['II_beta_HCG'].median())

    if 'AMHng_mL' in df.columns:
        df.loc[:, 'AMHng_mL'] = pd.to_numeric(df['AMHng_mL'], errors='coerce')
        df['AMHng_mL'] = df['AMHng_mL'].astype(float)
        df.loc[:, 'AMHng_mL'] = df['AMHng_mL'].fillna(df['AMHng_mL'].median())

    return df

In [4]:
# Apply preprocessing
original_clean = preprocess(original)
new_clean = preprocess(new)

# Ensure consistent columns across both
all_columns = list(set(original_clean.columns).union(set(new_clean.columns)))

# Align both dataframes to same columns, fill missing with NaN
original_aligned = original_clean.reindex(columns=all_columns)
new_aligned = new_clean.reindex(columns=all_columns)

# Concatenate datasets
combined_df = pd.concat([original_aligned, new_aligned], ignore_index=True)

In [5]:
# Separate features and target
# X = combined_df.drop(columns=['PCOS_Y_N'])
X = combined_df.drop(columns=['PCOS_Y_N'])
y = combined_df['PCOS_Y_N']

# Initialize KNNImputer (e.g., with 5 nearest neighbors)
knn_imputer = KNNImputer(n_neighbors=5)

X = pd.DataFrame(knn_imputer.fit_transform(X), columns=X.columns)

# Sanitize feature names
X.columns = [str(col).replace(' ', '_')
                        .replace('"', '')
                        .replace("'", '')
                        .replace('[', '')
                        .replace(']', '')
                        .replace('{', '')
                        .replace('}', '')
                        .replace(':', '')
                        .replace(',', '')
                        for col in X.columns]

In [6]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X = pd.DataFrame(X_scaled, columns=X.columns)

In [7]:
# LightGBM model
lgb_estimator = LGBMClassifier(n_estimators=100, random_state=8)

# Boruta Feature Selector
boruta_selector = BorutaPy(
    estimator=lgb_estimator,
    n_estimators=200,
    verbose=2,
    random_state=8
)

# Fit Boruta on training data
boruta_selector.fit(X.values, y.values)

# Get selected features
selected_features = X.columns[boruta_selector.support_].tolist()
print("Selected Features:", selected_features)

[LightGBM] [Info] Number of positive: 376, number of negative: 1165
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000678 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9300
[LightGBM] [Info] Number of data points in the train set: 1541, number of used features: 88
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.243997 -> initscore=-1.130887
[LightGBM] [Info] Start training from score -1.130887
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

In [8]:
X = X[selected_features]
X

,hair_growthY_N,Skin_darkening_Y_N,PimplesY_N,Menstrual_Irregularity,Follicle_No_R,Antral_Follicle_Count,Weight_gainY_N,FSH_LH,AMHng_mL,Follicle_No_L,BMI,Testosterone_Levelng_dL
0,-0.947571,-0.983225,-1.404234,0.655837,-1.123329,-0.416157,-1.085486,-0.112560,-0.906238,-1.004247,-1.339000,0.921325
1,-0.947571,-0.983225,-1.404234,-0.745242,-0.497294,-0.550558,-1.085486,-0.009492,-1.033911,-1.004247,-0.159445,-0.683265
2,-0.947571,-0.983225,1.415579,1.122863,2.632883,0.155050,-1.085486,-0.006151,0.171884,2.292470,-0.075192,0.412850
3,-0.947571,-0.983225,-1.404234,0.655837,-1.436347,-0.214554,-1.085486,-0.080175,-1.107204,-1.333919,0.851601,-0.414311
4,-0.947571,-0.983225,-1.404234,-0.278215,-0.810312,-0.248155,-1.085486,-0.054472,-0.861317,-1.004247,-1.170492,-0.028641
...,...,...,...,...,...,...,...,...,...,...,...,...
1536,-0.313525,0.200486,-1.404234,1.122863,-0.372087,0.927860,-1.085486,-0.079352,-0.137840,-0.674576,-1.528571,1.825620
1537,-0.313525,-0.391370,-0.276309,1.122863,-0.184276,-1.760174,0.454785,-0.001268,-0.600770,0.248505,0.683094,-1.584515
1538,-0.313525,-0.983225,-0.840271,-1.212268,-0.372087,1.767870,-0.058639,-0.072670,-0.145878,-0.147101,0.556713,-1.386606
1539,-0.313525,0.792341,-0.276309,-1.212268,-0.810312,-1.424169,1.481631,-0.127416,0.041847,-0.344904,0.346078,1.820545


In [9]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.15, stratify=y, shuffle=True, random_state=8)

In [10]:
def objective(trial):
    # Suggest hyperparameters to try
    param = {
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "num_leaves": trial.suggest_int("num_leaves", 20, 150),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 50),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 42,
        "objective": "binary",  # or "multiclass" if you're doing that
        "verbose": -1,
    }


    train_data = lgb.Dataset(x_train, label=y_train)
    valid_data = lgb.Dataset(x_test, label=y_test)

    gbm = lgb.train(param,
                    train_data,
                    valid_sets=[valid_data],
                    num_boost_round=1000)

    y_pred = gbm.predict(x_test)
    y_pred_labels = (y_pred > 0.5).astype(int)
    acc = accuracy_score(y_test, y_pred_labels)

    return 1.0 - acc  # Optuna minimizes, so 1 - accuracy


In [11]:
study = optuna.create_study(direction='minimize')  # Because we return 1 - accuracy
study.optimize(objective, n_trials=50)  # Try 50 combinations (or more!)

print("Best hyperparameters:")
print(study.best_params)

print("Best accuracy:")
print(1 - study.best_value)

[I 2025-05-31 20:55:03,321] A new study created in memory with name: no-name-93484e8c-535b-4c30-a8ce-9fa1c6071688
[I 2025-05-31 20:55:05,911] Trial 0 finished with value: 0.02155172413793105 and parameters: {'learning_rate': 0.27824933385244155, 'n_estimators': 961, 'max_depth': 10, 'num_leaves': 67, 'min_child_samples': 10, 'subsample': 0.9463053035712483, 'colsample_bytree': 0.8974852473388708, 'reg_alpha': 8.858593286812614e-06, 'reg_lambda': 6.195928129185884}. Best is trial 0 with value: 0.02155172413793105.
[I 2025-05-31 20:55:06,104] Trial 1 finished with value: 0.030172413793103425 and parameters: {'learning_rate': 0.06499839650388144, 'n_estimators': 102, 'max_depth': 7, 'num_leaves': 76, 'min_child_samples': 21, 'subsample': 0.6338884160191275, 'colsample_bytree': 0.7047972883524286, 'reg_alpha': 0.127983532974159, 'reg_lambda': 8.221271748863293e-07}. Best is trial 0 with value: 0.02155172413793105.
[I 2025-05-31 20:55:06,808] Trial 2 finished with value: 0.03448275862068961

Best hyperparameters:
{'learning_rate': 0.27824933385244155, 'n_estimators': 961, 'max_depth': 10, 'num_leaves': 67, 'min_child_samples': 10, 'subsample': 0.9463053035712483, 'colsample_bytree': 0.8974852473388708, 'reg_alpha': 8.858593286812614e-06, 'reg_lambda': 6.195928129185884}
Best accuracy:
0.978448275862069


In [12]:
train_data = lgb.Dataset(x_train, label=y_train)
valid_data = lgb.Dataset(x_test, label=y_test)

best_model = lgb.train(study.best_params,
                train_data,
                valid_sets=[valid_data],
                num_boost_round=1000)

# Predict
preds = best_model.predict(x_test)
preds = (preds > 0.5).astype(int)

# Compute final scores
acc = accuracy_score(y_test, preds)
precision = precision_score(y_test, preds, average='binary')
recall = recall_score(y_test, preds, average='binary')
f1 = f1_score(y_test, preds, average='binary')

print("\nFinal evaluation on test set:")
print(f"Accuracy: {acc * 100:.2f}")
print(f"Precision: {precision * 100:.2f}")
print(f"Recall: {recall * 100:.2f}")
print(f"F1 Score: {f1 * 100:.2f}")



Final evaluation on test set:
Accuracy: 96.12
Precision: 91.38
Recall: 92.98
F1 Score: 92.17
